In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests

def perform_proteomic_analysis(file_path):
    """Performs proteomic analysis step by step."""

    # Step 1: Data Loading and Preparation
    print("Step 1: Data Loading and Preparation")
    try:
        print("  1.1: Reading Excel file...")
        df = pd.read_excel(file_path, sheet_name=0)
        print("  1.2: Excel file loaded successfully.")
    except FileNotFoundError:
        print(f"  Error: File not found at {file_path}")
        return None
    except Exception as e:
        print(f"  An error occurred: {e}")
        return None

    # Step 2: Data Preprocessing and Calculation
    print("\nStep 2: Data Preprocessing and Calculation")
    print("  2.1: Identifying WT and KO columns...")
    # Corrected: Explicitly define the correct WT and KO columns
    wt_cols = ['WT1', 'WT2', 'WT3']
    ko_cols = ['KO1', 'KO2', 'KO3']
    print(f"    WT columns: {wt_cols}")
    print(f"    KO columns: {ko_cols}")

    print("  2.2: Calculating average intensities...")
    # Drop rows where any of the WT or KO intensity columns are NaN
    df_filtered = df[wt_cols + ko_cols].dropna().reset_index(drop=True) #reset index after dropna
    df['WT_avg'] = df_filtered[wt_cols].mean(axis=1)
    df['KO_avg'] = df_filtered[ko_cols].mean(axis=1)
    print("    Average intensities calculated.")

    print("  2.3: Calculating log2 fold change...")
    # Filter out rows with non-positive ratios
    ratio = df['KO_avg'] / df['WT_avg']
    df = df[ratio > 0].copy() #Filter the dataframe.
    df['log2FC'] = np.log2(df['KO_avg'] / df['WT_avg'])
    print("    log2 fold change calculated.")

    print("  2.4: Performing t-test...")
    if 'Student\'s T-test p-value KO all_WT all' not in df.columns:
        p_values = []
        for index, row in df[wt_cols+ko_cols].iterrows():
            wt_intensities = row[wt_cols]
            ko_intensities = row[ko_cols]
            t_stat, p_value = stats.ttest_ind(ko_intensities, wt_intensities, equal_var=False, nan_policy='omit')
            p_values.append(p_value)
        df['Student\'s T-test p-value KO all_WT all'] = p_values
        print("    T-test performed.")
    else:
        print("    T-test already performed (p-values column exists).")

    print("  2.5: Correcting p-values for multiple testing...")
    p_values = df['Student\'s T-test p-value KO all_WT all'].values
    reject, pvals_corrected, _, _ = multipletests(p_values, method='fdr_bh')
    df['Adjusted_p_value'] = pvals_corrected
    print("    Multiple testing correction applied.")

    print("  2.6: Calculating -log10(Adjusted_p_value)...")
    df['-log10(Adjusted_p_value)'] = -np.log10(df['Adjusted_p_value'])
    print("    -log10(Adjusted_p_value) calculated.")

    # Step 3: Visualization and Analysis
    #Volcano plot
    print("\nStep 3: Visualization and Analysis")
    print("  3.1: Generating volcano plot...")
    plt.figure(figsize=(10, 8))
    sns.scatterplot(x='log2FC', y='-log10(Adjusted_p_value)', data=df, alpha=0.7)
    plt.xlabel('log2 Fold Change (KO/WT)')
    plt.ylabel('-log10(Adjusted p-value)')
    plt.title('Volcano Plot')
    plt.axhline(y=-np.log10(0.05), color='r', linestyle='--', label='p = 0.05')
    plt.axvline(x=1, color='b', linestyle='--', label='log2FC = 1')
    plt.axvline(x=-1, color='b', linestyle='--')
    plt.legend()
    plt.show()
    print("    Volcano plot generated.")

    print("  3.2: Filtering significant proteins...")
    significant_df = df[(df['Adjusted_p_value'] < 0.05) & (abs(df['log2FC']) > 1)].copy()
    print("    Significant proteins filtered.")

    print("  3.3: Adding regulation column...")
    significant_df['regulation'] = 'unchanged'
    significant_df.loc[significant_df['log2FC'] > 1, 'regulation'] = 'up'
    significant_df.loc[significant_df['log2FC'] < -1, 'regulation'] = 'down'
    print("    Regulation column added.")

    print("  3.4: Outputting significant proteins...")
    print(significant_df[['Protein ID', 'Gene names', 'log2FC', 'Adjusted_p_value', 'regulation']].to_string())

    # Step 4: Return Results
    print("\nStep 4: Returning results.")
    return significant_df

# Example usage:
significant_proteins = perform_proteomic_analysis("41598_2023_42161_MOESM1_ESM.xlsx")

if significant_proteins is not None:
    # 1. Print the significant proteins to the console
    print("\nSignificant Proteins:")
    print(significant_proteins.to_string())  # Prints all rows

    # 2. Save the significant proteins to a CSV file
    significant_proteins.to_csv("significant_proteins.csv", index=False)
    print("\nSignificant proteins saved to significant_proteins.csv")

    # 3. Print the first few rows.
    print("\nFirst few rows of significant proteins:")
    print(significant_proteins.head())

    #4. Print the shape of the dataframe.
    print(f"\nSignificant proteins dataframe shape: {significant_proteins.shape}")

else:
    print("Proteomic analysis failed.")

In [ ]:
#volcano plot with significant proteins labeled
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def plot_volcano_with_labels(df):
    """Generates a volcano plot with significant proteins labeled."""
    
    plt.figure(figsize=(10, 8))
    
    # Plot all points
    sns.scatterplot(x='log2FC', y='-log10(Adjusted_p_value)', data=df, alpha=0.7)
    
    # Highlight significant points
    significant_df = df[(df['Adjusted_p_value'] < 0.05) & (abs(df['log2FC']) > 1)]
    sns.scatterplot(x='log2FC', y='-log10(Adjusted_p_value)', data=significant_df, color='red', alpha=0.8)
    
    # Add labels to significant points
    for _, row in significant_df.iterrows():
        plt.text(row['log2FC'], row['-log10(Adjusted_p_value)'], row['Gene names'], fontsize=8, ha='right')
    
    plt.xlabel('log2 Fold Change (KO/WT)')
    plt.ylabel('-log10(Adjusted p-value)')
    plt.title('Volcano Plot with Labeled Significant Proteins')
    plt.axhline(y=-np.log10(0.05), color='r', linestyle='--', label='p = 0.05')
    plt.axvline(x=1, color='b', linestyle='--', label='log2FC = 1')
    plt.axvline(x=-1, color='b', linestyle='--')
    plt.legend()
    plt.show()

# Example usage:
plot_volcano_with_labels(significant_proteins)


In [ ]:
#heatmap of the significant hits in WT and KO
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

def plot_heatmap(df, wt_cols, ko_cols):
    """Generates a heatmap of significant hits in WT and KO samples."""
    
    # Filter significant proteins
    significant_df = df[(df['Adjusted_p_value'] < 0.05) & (abs(df['log2FC']) > 1)]
    
    # Extract expression values for WT and KO columns
    heatmap_data = significant_df[wt_cols + ko_cols]
    
    # Normalize by row (Z-score transformation)
    heatmap_data = (heatmap_data - heatmap_data.mean(axis=1).values.reshape(-1, 1)) / heatmap_data.std(axis=1).values.reshape(-1, 1)
    
    # Plot heatmap
    plt.figure(figsize=(15, 16))
    sns.heatmap(heatmap_data, cmap='coolwarm', xticklabels=True, yticklabels=significant_df['Gene names'], linewidths=0.5)
    plt.xlabel("Samples")
    plt.ylabel("Significant Proteins")
    plt.title("Heatmap of Significant Proteins in WT and KO")
    plt.show()

# Example usage:
plot_heatmap(significant_proteins, ['1 WT', '4 WT', '6 WT', '10 WT', '11 WT', '12 WT'], ['2 KO', '3 KO', '5 KO', '7 KO', '8 KO', '9 KO'])


In [ ]:
#Clustering Analysis
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
def hierarchical_clustering(df, wt_cols, ko_cols):
    """Performs hierarchical clustering and plots the dendrogram."""
    expression_data = df[wt_cols + ko_cols]
    linked = linkage(expression_data, method='ward')
    plt.figure(figsize=(10, 6))
    dendrogram(linked, labels=df['Gene names'].values, leaf_rotation=90, leaf_font_size=8)
    plt.title("Hierarchical Clustering Dendrogram")
    plt.xlabel("Proteins")
    plt.ylabel("Distance")
    plt.show()

def kmeans_clustering(df, wt_cols, ko_cols, n_clusters=3):
    """Performs K-Means clustering and adds cluster labels."""
    expression_data = df[wt_cols + ko_cols]
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    df['Cluster'] = kmeans.fit_predict(expression_data)
    
    plt.figure(figsize=(8, 6))
    scatter = sns.scatterplot(x=expression_data.iloc[:, 0], y=expression_data.iloc[:, 1], hue=df['Cluster'], palette='viridis')
    
    # Label each point in the scatter plot
    for i, gene in enumerate(df['Gene names']):
        scatter.text(expression_data.iloc[i, 0], expression_data.iloc[i, 1], gene, fontsize=8, ha='right', va='bottom')
    
    plt.xlabel(wt_cols[0])
    plt.ylabel(ko_cols[0])
    plt.title("K-Means Clustering")
    plt.legend(title="Cluster")
    plt.show()

    
def pca_analysis(df, wt_cols, ko_cols):
    """Performs PCA and plots the first two principal components."""
    expression_data = df[wt_cols + ko_cols]
    
    # Perform PCA
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(expression_data)
    df['PC1'], df['PC2'] = pca_result[:, 0], pca_result[:, 1]
    
    # Create a new column for group categorization (WT vs KO) based on rows
    group_labels = ['WT' if col in wt_cols else 'KO' for col in wt_cols + ko_cols]
    df['Group'] = group_labels * (df.shape[0] // len(group_labels))  # Ensure correct length

    plt.figure(figsize=(8, 6))
    scatter = sns.scatterplot(data=df, x='PC1', y='PC2', hue='Group', palette=['blue', 'red'])
    
    # Label each point in the PCA plot
    for i, gene in enumerate(df['Gene names']):
        scatter.text(df.iloc[i]['PC1'], df.iloc[i]['PC2'], gene, fontsize=8, ha='right', va='bottom')
    
    plt.xlabel("Principal Component 1")
    plt.ylabel("Principal Component 2")
    plt.title("PCA of Proteomic Data")
    plt.show()

    
# Example usage:
hierarchical_clustering(significant_proteins, ['1 WT', '4 WT', '6 WT', '10 WT', '11 WT', '12 WT'], ['2 KO', '3 KO', '5 KO', '7 KO', '8 KO', '9 KO'])
kmeans_clustering(significant_proteins, ['1 WT', '4 WT', '6 WT', '10 WT', '11 WT', '12 WT'], ['2 KO', '3 KO', '5 KO', '7 KO', '8 KO', '9 KO'], n_clusters=3)
pca_analysis(significant_proteins, ['1 WT', '4 WT', '6 WT', '10 WT', '11 WT', '12 WT'], ['2 KO', '3 KO', '5 KO', '7 KO', '8 KO', '9 KO'])

In [ ]:
#Gene Ontology (GO) Enrichment

import gseapy as gp

def functional_enrichment_analysis(df, gene_column, output_csv_path):
    """Performs GO enrichment analysis on a list of genes and saves the results to a CSV file."""
    # Extract the list of genes
    gene_list = df[gene_column].dropna().tolist()
    
    # Run GO enrichment analysis using Enrichr (GO Biological Process 2018)
    enrichment_results = gp.enrichr(gene_list=gene_list, gene_sets='GO_Biological_Process_2018', 
                                    organism='Human', outdir=None)
    
    # Get the results
    enriched_df = enrichment_results.results
    print("All Enriched GO Terms:")
    print(enriched_df)  # Prints all the enriched terms
    
    # Save the results to a CSV file
    enriched_df.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")
    
    return enriched_df

# Example usage with the path for the CSV output
functional_enrichment_analysis(significant_proteins, 'Gene names', 'enriched_go_terms.csv')
